In [ ]:
import spacy

from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_dir_path = '/content/drive/MyDrive/ML Project/data/image/'
train_tabular = pd.read_csv(data_dir_path + 'train_image_sample')
test_tabular = pd.read_csv( data_dir_path + 'test_image_sample')
print(train_tabular.shape)
print(test_tabular.shape)

(222532, 43)
(55635, 43)


In [ ]:
train_tabular_X = train_tabular.drop(['title','description', 'item_id', 'user_id', 'item_seq_number', 'activation_date', 'image',
        'image_top_1', 'deal_probability'], axis=1)
test_tabular_X = test_tabular.drop(['title','description', 'item_id', 'user_id', 'item_seq_number', 'activation_date', 'image',
        'image_top_1', 'deal_probability'], axis=1)
train_y = train_tabular['deal_probability']
test_y = test_tabular['deal_probability']

train_tabular_image_id = train_tabular['image']
test_tabular_image_id = test_tabular['image']

categorical_features = train_tabular_X.select_dtypes(include=['object', 'category']).columns.tolist()
for col in categorical_features:
    train_tabular_X[col] = train_tabular_X[col].astype('category')
    test_tabular_X[col] = test_tabular_X[col].astype('category')

print(train_tabular_X.shape)
print(test_tabular_X.shape)

(222532, 34)
(55635, 34)


In [ ]:
train_image_X = pd.read_csv(data_dir_path + 'train_image_svd.csv')
test_image_X = pd.read_csv(data_dir_path + 'test_image_svd.csv')

train_image_image_id = train_image_X['image']
test_image_image_id = test_image_X['image']

train_image_X = train_image_X.drop(['image'], axis=1)
test_image_X = test_image_X.drop(['image'], axis=1)

print(train_image_X.shape)
print(test_image_X.shape)

(222532, 500)
(55635, 500)


As shown here, the image ids are aligned already, so no need to manually re-order them.

**Only re-order if this cell raises an exception.**

In [ ]:
assert (train_tabular_image_id == train_image_image_id).any()
assert (test_tabular_image_id == test_image_image_id).any()

# Helper function

In [ ]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, root_mean_squared_error
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import lightgbm as lgb
from sklearn.decomposition import TruncatedSVD

In [ ]:
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'learning_rate': 0.02,
    'num_leaves': 512,
    'max_depth': 20,
    'bagging_seed': 42,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'feature_fraction': 0.8,
    'lambda_l1': 2.25,
    'lambda_l2': 0.015,
    'verbose': -1
}

In [ ]:
def fit_lgb(
        train_X, train_y, test_X, test_y, categorical_features,
        params,
        num_boost_round=3000,
        early_stopping_rounds=50,
        eval_period=40,
    ):

    # validation set for early stopping
    train_X_split, val_X_split, train_y_split, val_y_split = train_test_split(
        train_X, train_y, test_size=0.2, random_state=42, shuffle=True
    )
    lgb_train = lgb.Dataset(
        train_X_split, label=train_y_split, categorical_feature=categorical_features
    )
    lgb_val = lgb.Dataset(
        val_X_split, label=val_y_split, categorical_feature=categorical_features,
        reference=lgb_train
    )

    callbacks = [
        lgb.early_stopping(stopping_rounds=early_stopping_rounds),
        lgb.log_evaluation(period=eval_period)
    ]

    model = lgb.train(
        params,
        lgb_train,
        num_boost_round=num_boost_round,
        valid_sets=[lgb_train, lgb_val],
        callbacks=callbacks
    )

    preds = model.predict(test_X, num_iteration=model.best_iteration)
    rmse = root_mean_squared_error(test_y, preds)
    print(f"RMSE: {rmse}")

    return model

In [ ]:
def get_feature_importance(model):
    feature_importances = pd.DataFrame({
        'feature': model.feature_name(),
        'importance': model.feature_importance()
    }).sort_values(by='importance', ascending=False)

    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        print(feature_importances)
    return feature_importances

# Tune the tabular tree using subset data

In [ ]:
# !pip install optuna

In [ ]:
import optuna

def tune_params(train_X, train_y, categorical_features):
    train_data, val_data, train_label, val_label = train_test_split(
        train_X, train_y, test_size=0.2, random_state=42)

    def objective(trial):
        params = {
            'objective': 'regression',
            'metric': 'rmse',
            'boosting_type': 'gbdt',
            'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
            'num_leaves': trial.suggest_int('num_leaves', 32, 512),
            'max_depth': trial.suggest_int('max_depth', 3, 25),
            'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-3, 10.0),
            'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-3, 10.0),
            'verbose': -1,
            'bagging_seed': 42,
            'bagging_fraction': 0.8,
            'bagging_freq': 5,
            'feature_fraction': 0.8,
        }

        dtrain = lgb.Dataset(train_data, label=train_label, categorical_feature=categorical_features)
        dval = lgb.Dataset(val_data, label=val_label, reference=dtrain, categorical_feature=categorical_features)

        callbacks = [
            lgb.early_stopping(stopping_rounds=50),
            lgb.log_evaluation(period=100)
        ]
        model = lgb.train(
            params, dtrain, valid_sets=[dtrain, dval], callbacks=callbacks,
            num_boost_round=1000
        )
        preds = model.predict(val_data)
        return root_mean_squared_error(val_label, preds)

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=50, timeout=5400)

    print("Best parameters:", study.best_params)

In [ ]:
tune_params(train_tabular_X, train_y, categorical_features)

[I 2024-12-03 03:00:41,765] A new study created in memory with name: no-name-f426d772-55d4-4158-942c-53ce8b90884a


Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 0.222412	valid_1's rmse: 0.233096
Early stopping, best iteration is:
[145]	training's rmse: 0.219453	valid_1's rmse: 0.232665


[I 2024-12-03 03:00:49,498] Trial 0 finished with value: 0.23266465409837425 and parameters: {'learning_rate': 0.021043176935301448, 'num_leaves': 509, 'max_depth': 9, 'lambda_l1': 0.0012477489985522044, 'lambda_l2': 3.0402529741294115}. Best is trial 0 with value: 0.23266465409837425.


Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 0.219802	valid_1's rmse: 0.233331
Early stopping, best iteration is:
[56]	training's rmse: 0.223117	valid_1's rmse: 0.233263


[I 2024-12-03 03:00:52,421] Trial 1 finished with value: 0.2332630326758449 and parameters: {'learning_rate': 0.0908069776069154, 'num_leaves': 85, 'max_depth': 7, 'lambda_l1': 0.022917989645981712, 'lambda_l2': 0.0023586829378410253}. Best is trial 0 with value: 0.23266465409837425.


...

Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 0.240261	valid_1's rmse: 0.241201
[200]	training's rmse: 0.235775	valid_1's rmse: 0.237382
[300]	training's rmse: 0.23404	valid_1's rmse: 0.236284
[400]	training's rmse: 0.232954	valid_1's rmse: 0.235722
[500]	training's rmse: 0.232138	valid_1's rmse: 0.235355
[600]	training's rmse: 0.23155	valid_1's rmse: 0.235176
[700]	training's rmse: 0.230964	valid_1's rmse: 0.234986
[800]	training's rmse: 0.230494	valid_1's rmse: 0.234884
[900]	training's rmse: 0.230031	valid_1's rmse: 0.234709
[1000]	training's rmse: 0.229626	valid_1's rmse: 0.2346
Did not meet early stopping. Best iteration is:
[1000]	training's rmse: 0.229626	valid_1's rmse: 0.2346


[I 2024-12-03 03:11:09,291] Trial 48 finished with value: 0.23459964342227888 and parameters: {'learning_rate': 0.011163599892692139, 'num_leaves': 511, 'max_depth': 3, 'lambda_l1': 0.002673813914922008, 'lambda_l2': 2.2471283898579726}. Best is trial 44 with value: 0.23214882260549774.


Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 0.218606	valid_1's rmse: 0.233536
[200]	training's rmse: 0.206411	valid_1's rmse: 0.23222
Early stopping, best iteration is:
[169]	training's rmse: 0.209212	valid_1's rmse: 0.23218


[I 2024-12-03 03:11:25,640] Trial 49 finished with value: 0.2321798129514878 and parameters: {'learning_rate': 0.014848458846482395, 'num_leaves': 453, 'max_depth': 21, 'lambda_l1': 0.004858230357391291, 'lambda_l2': 1.6892354956820483}. Best is trial 44 with value: 0.23214882260549774.


Best parameters: {'learning_rate': 0.014223644707613666, 'num_leaves': 512, 'max_depth': 19, 'lambda_l1': 0.004071154648893681, 'lambda_l2': 1.4099886344987713}


In [ ]:
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'learning_rate': 0.015,
    'num_leaves': 512,
    'max_depth': 19,
    'bagging_seed': 42,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'feature_fraction': 0.8,
    'lambda_l1': 0.004,
    'lambda_l2': 1.41,
    'verbose': -1
}

In [ ]:
model_tabular = fit_lgb(
    train_tabular_X, train_y,
    test_tabular_X, test_y,
    categorical_features=categorical_features,
    params=params,
)

Training until validation scores don't improve for 50 rounds
[40]	training's rmse: 0.234383	valid_1's rmse: 0.241233
[80]	training's rmse: 0.221675	valid_1's rmse: 0.234866
[120]	training's rmse: 0.214027	valid_1's rmse: 0.232811
[160]	training's rmse: 0.208875	valid_1's rmse: 0.232367
[200]	training's rmse: 0.205061	valid_1's rmse: 0.232388
Early stopping, best iteration is:
[170]	training's rmse: 0.207704	valid_1's rmse: 0.232313
RMSE: 0.23201362973381603


# LightGBM with image only

In [ ]:
model_image_only = fit_lgb(
    train_image_X, train_y,
    test_image_X, test_y,
    categorical_features=[],
    params=params,
)

Training until validation scores don't improve for 50 rounds
[40]	training's rmse: 0.240442	valid_1's rmse: 0.250389
[80]	training's rmse: 0.227216	valid_1's rmse: 0.246379
[120]	training's rmse: 0.216999	valid_1's rmse: 0.244656
[160]	training's rmse: 0.208308	valid_1's rmse: 0.243891
[200]	training's rmse: 0.200535	valid_1's rmse: 0.243424
[240]	training's rmse: 0.193387	valid_1's rmse: 0.243142
[280]	training's rmse: 0.186726	valid_1's rmse: 0.242976
[320]	training's rmse: 0.18046	valid_1's rmse: 0.242812
[360]	training's rmse: 0.174563	valid_1's rmse: 0.242708
[400]	training's rmse: 0.168932	valid_1's rmse: 0.242645
[440]	training's rmse: 0.163605	valid_1's rmse: 0.242575
[480]	training's rmse: 0.1585	valid_1's rmse: 0.242511
[520]	training's rmse: 0.153647	valid_1's rmse: 0.242497
[560]	training's rmse: 0.148909	valid_1's rmse: 0.242491
[600]	training's rmse: 0.144334	valid_1's rmse: 0.242462
[640]	training's rmse: 0.139979	valid_1's rmse: 0.242452
[680]	training's rmse: 0.13583	v

# LightGBM with tabular and image

重新tune了, 但是作用不大, 大概0.0001 (0.23058407724168462 到 0.23048984669350972)

In [ ]:
train_merged_X = pd.concat([train_tabular_X, train_image_X], axis=1)
test_merged_X = pd.concat([test_tabular_X, test_image_X], axis=1)

In [ ]:
tune_params(train_merged_X, train_y, categorical_features)

[I 2024-12-03 03:45:42,919] A new study created in memory with name: no-name-a7c616ea-c15d-496a-81ad-a248862ad48e


Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 0.208326	valid_1's rmse: 0.231579
[200]	training's rmse: 0.190566	valid_1's rmse: 0.231301
Early stopping, best iteration is:
[166]	training's rmse: 0.1959	valid_1's rmse: 0.23121


[I 2024-12-03 03:47:27,891] Trial 0 finished with value: 0.23120950316652814 and parameters: {'learning_rate': 0.02429618726364619, 'num_leaves': 217, 'max_depth': 22, 'lambda_l1': 0.0020150120220779116, 'lambda_l2': 0.0019338570570551616}. Best is trial 0 with value: 0.23120950316652814.


Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 0.193243	valid_1's rmse: 0.231805
Early stopping, best iteration is:
[63]	training's rmse: 0.202784	valid_1's rmse: 0.23167


[I 2024-12-03 03:48:16,716] Trial 1 finished with value: 0.23167041366684796 and parameters: {'learning_rate': 0.05742061349442545, 'num_leaves': 226, 'max_depth': 13, 'lambda_l1': 0.5762197649425199, 'lambda_l2': 0.10693448076355995}. Best is trial 0 with value: 0.23120950316652814.


...

Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 0.219756	valid_1's rmse: 0.23511
[200]	training's rmse: 0.202673	valid_1's rmse: 0.231312
[300]	training's rmse: 0.191205	valid_1's rmse: 0.230658
[400]	training's rmse: 0.182022	valid_1's rmse: 0.2306
Early stopping, best iteration is:
[360]	training's rmse: 0.18555	valid_1's rmse: 0.230565


[I 2024-12-03 05:11:54,361] Trial 35 finished with value: 0.23056504563552008 and parameters: {'learning_rate': 0.011285545573319579, 'num_leaves': 326, 'max_depth': 21, 'lambda_l1': 0.35056474452290404, 'lambda_l2': 0.26874865688152816}. Best is trial 35 with value: 0.23056504563552008.


Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 0.218229	valid_1's rmse: 0.234446
[200]	training's rmse: 0.201382	valid_1's rmse: 0.231279
[300]	training's rmse: 0.190815	valid_1's rmse: 0.23083
[400]	training's rmse: 0.182172	valid_1's rmse: 0.230739
[500]	training's rmse: 0.174785	valid_1's rmse: 0.230681
[600]	training's rmse: 0.16795	valid_1's rmse: 0.230646
Early stopping, best iteration is:
[563]	training's rmse: 0.170404	valid_1's rmse: 0.230637


[I 2024-12-03 05:16:15,369] Trial 36 finished with value: 0.23063737822442745 and parameters: {'learning_rate': 0.012461010729162982, 'num_leaves': 324, 'max_depth': 18, 'lambda_l1': 0.7876142008786086, 'lambda_l2': 0.15536633668631308}. Best is trial 35 with value: 0.23056504563552008.


Best parameters: {'learning_rate': 0.011285545573319579, 'num_leaves': 326, 'max_depth': 21, 'lambda_l1': 0.35056474452290404, 'lambda_l2': 0.26874865688152816}


In [ ]:
merged_params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'learning_rate': 0.011,
    'num_leaves': 326,
    'max_depth': 21,
    'bagging_seed': 42,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'feature_fraction': 0.8,
    'lambda_l1': 0.3505,
    'lambda_l2': 0.2687,
    'verbose': -1
}

In [ ]:
model_merged = fit_lgb(
    train_merged_X, train_y,
    test_merged_X, test_y,
    categorical_features=categorical_features,
    params=merged_params,
)

Training until validation scores don't improve for 50 rounds
[40]	training's rmse: 0.238625	valid_1's rmse: 0.244821
[80]	training's rmse: 0.225265	valid_1's rmse: 0.237428
[120]	training's rmse: 0.216111	valid_1's rmse: 0.233972
[160]	training's rmse: 0.209147	valid_1's rmse: 0.232279
[200]	training's rmse: 0.20334	valid_1's rmse: 0.231475
[240]	training's rmse: 0.198271	valid_1's rmse: 0.231042
[280]	training's rmse: 0.193877	valid_1's rmse: 0.230792
[320]	training's rmse: 0.189968	valid_1's rmse: 0.230718
[360]	training's rmse: 0.186301	valid_1's rmse: 0.230624
[400]	training's rmse: 0.18289	valid_1's rmse: 0.230617
Early stopping, best iteration is:
[380]	training's rmse: 0.184608	valid_1's rmse: 0.230599
RMSE: 0.23048984669350972


# Tabular + Text + Image Tree

In [ ]:
# keep only the svd embedding columns, and rows that are in the image subsample

# be very careful using merge: the one calling merge keeps the order!
train_text_X = pd.read_csv(data_dir_path + '../train_embeddings_fasttext_svd.csv')
train_text_X = train_tabular[["item_id", "user_id"]].merge(train_text_X, on=["item_id", "user_id"], how="inner")
train_text_X = train_text_X.drop(columns = train_tabular.columns.tolist(), errors='ignore')

test_text_X = pd.read_csv(data_dir_path + '../test_embeddings_fasttext_svd.csv')
test_text_X = test_tabular[["item_id", "user_id"]].merge(test_text_X, on=["item_id", "user_id"], how="inner")
test_text_X = test_text_X.drop(columns = test_tabular.columns.tolist(), errors='ignore')

print(train_text_X.shape)
print(test_text_X.shape)

(222532, 42)
(55635, 42)


In [ ]:
train_merged_all_X = pd.concat([train_tabular_X, train_image_X, train_text_X], axis=1)
test_merged_all_X = pd.concat([test_tabular_X, test_image_X, test_text_X], axis=1)
print(train_merged_all_X.shape)
print(test_merged_all_X.shape)

(222532, 576)
(55635, 576)


In [ ]:
tune_params(train_merged_all_X, train_y, categorical_features)

[I 2024-12-03 06:17:52,743] A new study created in memory with name: no-name-b104fea7-ca66-49fd-af7c-ba4ba32ecb44


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[42]	training's rmse: 0.208141	valid_1's rmse: 0.231413


[I 2024-12-03 06:18:45,037] Trial 0 finished with value: 0.23141323183241544 and parameters: {'learning_rate': 0.07677255085172037, 'num_leaves': 153, 'max_depth': 25, 'lambda_l1': 0.1974996555862863, 'lambda_l2': 0.002664849029414945}. Best is trial 0 with value: 0.23141323183241544.


Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 0.231551	valid_1's rmse: 0.234627
[200]	training's rmse: 0.228062	valid_1's rmse: 0.233303
[300]	training's rmse: 0.225772	valid_1's rmse: 0.232789
[400]	training's rmse: 0.223609	valid_1's rmse: 0.23239
[500]	training's rmse: 0.221827	valid_1's rmse: 0.232235
[600]	training's rmse: 0.22024	valid_1's rmse: 0.232056
[700]	training's rmse: 0.218937	valid_1's rmse: 0.231909
[800]	training's rmse: 0.217912	valid_1's rmse: 0.23181
[900]	training's rmse: 0.216947	valid_1's rmse: 0.231771
Early stopping, best iteration is:
[923]	training's rmse: 0.21677	valid_1's rmse: 0.231758


[I 2024-12-03 06:20:04,402] Trial 1 finished with value: 0.2317582827053883 and parameters: {'learning_rate': 0.033054421323574064, 'num_leaves': 307, 'max_depth': 4, 'lambda_l1': 9.859363287549712, 'lambda_l2': 9.520569722083962}. Best is trial 0 with value: 0.23141323183241544.


...

Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 0.191051	valid_1's rmse: 0.230716
[200]	training's rmse: 0.167729	valid_1's rmse: 0.230313
Early stopping, best iteration is:
[172]	training's rmse: 0.17304	valid_1's rmse: 0.230257


[I 2024-12-03 07:45:54,851] Trial 29 finished with value: 0.23025740954581936 and parameters: {'learning_rate': 0.025337748984772628, 'num_leaves': 457, 'max_depth': 20, 'lambda_l1': 0.23689578974916353, 'lambda_l2': 2.467169601411372}. Best is trial 26 with value: 0.22974859488345928.


Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 0.225098	valid_1's rmse: 0.234627
[200]	training's rmse: 0.215031	valid_1's rmse: 0.231708
[300]	training's rmse: 0.210994	valid_1's rmse: 0.231313
[400]	training's rmse: 0.207765	valid_1's rmse: 0.23116
[500]	training's rmse: 0.204838	valid_1's rmse: 0.231041
[600]	training's rmse: 0.202115	valid_1's rmse: 0.231014
[700]	training's rmse: 0.199704	valid_1's rmse: 0.230922
[800]	training's rmse: 0.19742	valid_1's rmse: 0.230867
Early stopping, best iteration is:
[837]	training's rmse: 0.196592	valid_1's rmse: 0.230858


[I 2024-12-03 07:48:23,587] Trial 30 finished with value: 0.23085777838273136 and parameters: {'learning_rate': 0.013505909577059782, 'num_leaves': 265, 'max_depth': 9, 'lambda_l1': 2.4901407895606282, 'lambda_l2': 0.0042050996460699765}. Best is trial 26 with value: 0.22974859488345928.


Best parameters: {'learning_rate': 0.016900255014911943, 'num_leaves': 378, 'max_depth': 19, 'lambda_l1': 1.2447563180739318, 'lambda_l2': 1.8717922345065867}


In [ ]:
merged_all_params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'learning_rate': 0.017,
    'num_leaves': 378,
    'max_depth': 19,
    'bagging_seed': 42,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'feature_fraction': 0.8,
    'lambda_l1': 1.2447563180739318,
    'lambda_l2': 1.8717922345065867,
    'verbose': -1
}

In [ ]:
merged_all_model = fit_lgb(
    train_merged_all_X, train_y,
    test_merged_all_X, test_y,
    categorical_features=categorical_features,
    params=merged_all_params,
)

Training until validation scores don't improve for 50 rounds
[40]	training's rmse: 0.230461	valid_1's rmse: 0.239956
[80]	training's rmse: 0.214825	valid_1's rmse: 0.233482
[120]	training's rmse: 0.204295	valid_1's rmse: 0.231299
[160]	training's rmse: 0.196023	valid_1's rmse: 0.230493
[200]	training's rmse: 0.190018	valid_1's rmse: 0.230213
[240]	training's rmse: 0.184983	valid_1's rmse: 0.230137
[280]	training's rmse: 0.180301	valid_1's rmse: 0.230039
[320]	training's rmse: 0.175782	valid_1's rmse: 0.230026
[360]	training's rmse: 0.171614	valid_1's rmse: 0.230023
[400]	training's rmse: 0.167691	valid_1's rmse: 0.230024
[440]	training's rmse: 0.163956	valid_1's rmse: 0.229978
[480]	training's rmse: 0.16037	valid_1's rmse: 0.22995
[520]	training's rmse: 0.156981	valid_1's rmse: 0.229936
[560]	training's rmse: 0.153703	valid_1's rmse: 0.229906
Early stopping, best iteration is:
[547]	training's rmse: 0.15465	valid_1's rmse: 0.229883
RMSE: 0.22980301797838515


# Stacking: poor results, attempt abandoned

## Stack: Image-only Pred to Tabular Tree

In [ ]:
train_stack_X = train_tabular_X.copy()
test_stack_X = test_tabular_X.copy()

train_stack_X['image_pred'] = model_image_only.predict(train_image_X)
test_stack_X['image_pred'] = model_image_only.predict(test_image_X)

In [ ]:
model_stack = fit_lgb(
    train_stack_X, train_y,
    test_stack_X, test_y,
    categorical_features=categorical_features,
    params=params,
)

Training until validation scores don't improve for 50 rounds
[40]	training's rmse: 0.14065	valid_1's rmse: 0.245221
[80]	training's rmse: 0.0890579	valid_1's rmse: 0.247007
Early stopping, best iteration is:
[50]	training's rmse: 0.125346	valid_1's rmse: 0.244718
RMSE: 0.24423304728063672


## Stack: Image Pred and Tabular Pred to Logistic reg

In [ ]:
model_tabular = fit_lgb(
    train_tabular_X, train_y,
    test_tabular_X, test_y,
    categorical_features=categorical_features,
    params=params,
)

Training until validation scores don't improve for 50 rounds
[40]	training's rmse: 0.231726	valid_1's rmse: 0.238728
[80]	training's rmse: 0.220294	valid_1's rmse: 0.233625
[120]	training's rmse: 0.214177	valid_1's rmse: 0.232531
[160]	training's rmse: 0.210606	valid_1's rmse: 0.232504
Early stopping, best iteration is:
[142]	training's rmse: 0.211826	valid_1's rmse: 0.232428
RMSE: 0.2321422109231478


In [ ]:
train_stack2_x = pd.DataFrame({
    'tabular_pred': model_tabular.predict(train_tabular_X),
    'image_pred': model_image_only.predict(train_image_X)
})
test_stack2_x = pd.DataFrame({
    'tabular_pred': model_tabular.predict(test_tabular_X),
    'image_pred': model_image_only.predict(test_image_X)
})

In [ ]:
params

{'objective': 'regression',
 'metric': 'rmse',
 'boosting_type': 'gbdt',
 'learning_rate': 0.02,
 'num_leaves': 512,
 'max_depth': 20,
 'bagging_seed': 42,
 'bagging_fraction': 0.8,
 'bagging_freq': 5,
 'feature_fraction': 0.8,
 'lambda_l1': 2.25,
 'lambda_l2': 0.015,
 'verbose': -1}

In [ ]:
model_stack2 = fit_lgb(
    train_stack2_x, train_y,
    test_stack2_x, test_y,
    categorical_features=[],
    params={
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'learning_rate': 0.001,
        'num_leaves': 64,
        'max_depth': 5,
    },
)

Training until validation scores don't improve for 50 rounds
[40]	training's rmse: 0.251634	valid_1's rmse: 0.258936
[80]	training's rmse: 0.242954	valid_1's rmse: 0.257233
[120]	training's rmse: 0.234656	valid_1's rmse: 0.255728
[160]	training's rmse: 0.226724	valid_1's rmse: 0.254416
[200]	training's rmse: 0.219146	valid_1's rmse: 0.253282
[240]	training's rmse: 0.211909	valid_1's rmse: 0.252311
[280]	training's rmse: 0.205001	valid_1's rmse: 0.251485
[320]	training's rmse: 0.198409	valid_1's rmse: 0.2508
[360]	training's rmse: 0.192119	valid_1's rmse: 0.250236
[400]	training's rmse: 0.186122	valid_1's rmse: 0.249781
[440]	training's rmse: 0.180409	valid_1's rmse: 0.249425
[480]	training's rmse: 0.174968	valid_1's rmse: 0.249155
[520]	training's rmse: 0.169789	valid_1's rmse: 0.248962
[560]	training's rmse: 0.164863	valid_1's rmse: 0.248838
[600]	training's rmse: 0.160179	valid_1's rmse: 0.248771
[640]	training's rmse: 0.155729	valid_1's rmse: 0.24876
Early stopping, best iteration i